In [ ]:
#The book is to predict whether a loan should be granted to a customer or not.

#Log Reg Caveats
#Great for quick, fast performance on any given problem.

#When to USE IT
#Binary Target Variable
#Only for clean, well behaved data.

#DO NOT USE IT
#Not for continuous variable
#Not for massive amounts of data.
#Should not be used on ill behaving data - Outliers, Skewed Features, Missing Values, Really Complex relationships
#Rarely be the best.

In [43]:
#Import relevant libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import sklearn

from pandas import Series, DataFrame
from pylab import rcParams
from sklearn import preprocessing

from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict

from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score

In [2]:
%matplotlib inline
rcParams['figure.figsize'] = 5, 4 #plotting params
sb.set_style('whitegrid')

In [12]:
bank = pd.read_csv('/Users/kashs/Datasets/bank/bank-full.csv', sep = ';')
bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [13]:
bank.info()

#bank.isnull().sum()
#bank.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
age          45211 non-null int64
job          45211 non-null object
marital      45211 non-null object
education    45211 non-null object
default      45211 non-null object
balance      45211 non-null int64
housing      45211 non-null object
loan         45211 non-null object
contact      45211 non-null object
day          45211 non-null int64
month        45211 non-null object
duration     45211 non-null int64
campaign     45211 non-null int64
pdays        45211 non-null int64
previous     45211 non-null int64
poutcome     45211 non-null object
y            45211 non-null object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [14]:
#Let's see how many converted vs how many that did not.
bank['y'].value_counts()

no     39922
yes     5289
Name: y, dtype: int64

In [15]:
bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


# y-target variable

In [16]:
#Since the data is clean and this project is about Logistic Regression classifier, let's dummify the variables.
#Make new columns and append to DF.

#y target variable - mapping 1 for Yes and 0 for No.
bank['y']=bank['y'].map({'yes':1,'no':0})

In [18]:
bank

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,0
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,0
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,0
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,0
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,1
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,1
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,1
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,0


In [19]:
#Dropping columns
bank.drop(['poutcome','day','month'], axis=1, inplace=True)

In [20]:
#Getting Dummies for all faetures.

#Split into predictors and response variables.
X = bank.drop('y', axis=1)
y = bank['y']

In [21]:
#Dummifying
X = pd.get_dummies(X)
X

,age,balance,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,education_unknown,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone,contact_unknown
0,58,2143,261,1,-1,0,0,0,0,0,...,0,1,0,0,1,1,0,0,0,1
1,44,29,151,1,-1,0,0,0,0,0,...,0,1,0,0,1,1,0,0,0,1
2,33,2,76,1,-1,0,0,0,1,0,...,0,1,0,0,1,0,1,0,0,1
3,47,1506,92,1,-1,0,0,1,0,0,...,1,1,0,0,1,1,0,0,0,1
4,33,1,198,1,-1,0,0,0,0,0,...,1,1,0,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,825,977,3,-1,0,0,0,0,0,...,0,1,0,1,0,1,0,1,0,0
45207,71,1729,456,2,-1,0,0,0,0,0,...,0,1,0,1,0,1,0,1,0,0
45208,72,5715,1127,5,184,3,0,0,0,0,...,0,1,0,1,0,1,0,1,0,0
45209,57,668,508,4,-1,0,0,1,0,0,...,0,1,0,1,0,1,0,0,1,0


In [22]:
#Scaling features.
#Since the data is not standardized, it needs to be scaled to the same level.
scaler = preprocessing.MinMaxScaler()#(feature_range=(0,1)) can choose range too. can get constant values 
X = scaler.fit_transform(X)

# Model Ready : Applying Logistic Regression

In [23]:
#Split the dataset.

#Set aside 25% of the data for test. Keep 75% for training.
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size=.75, random_state = 200)

# Check Parameters Before Building The Model

In [24]:
def print_results(results):
    print('BEST PARAMS: {}\n'.format(results.best_params_))

    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print('{} (+/-{}) for {}'.format(round(mean, 3), round(std * 2, 3), params))

In [25]:
logreg = LogisticRegression()
parameters = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}

cv = GridSearchCV(logreg, parameters, cv=5)
cv.fit(X_train, y_train.ravel())

print_results(cv)

#When C is close to 1, it's performing accuracy is at 89.3%

/Users/kashs/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/kashs/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear

/Users/kashs/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/kashs/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear

BEST PARAMS: {'C': 1000}

0.881 (+/-0.0) for {'C': 0.001}
0.881 (+/-0.0) for {'C': 0.01}
0.882 (+/-0.003) for {'C': 0.1}
0.889 (+/-0.005) for {'C': 1}
0.891 (+/-0.003) for {'C': 10}
0.891 (+/-0.005) for {'C': 100}
0.891 (+/-0.005) for {'C': 1000}


/Users/kashs/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [26]:
cv.best_estimator_

LogisticRegression(C=1000, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

# Build and Fit the Model

In [27]:
#Build and fit the model.
logreg = LogisticRegression(C=1, solver='liblinear')
logreg.fit(X_train,y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [28]:
y_pred=logreg.predict(X_test)

In [29]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

#Actual predictions: 29602,672

array([[29549,   417],
       [ 3285,   658]])

In [30]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8908254445722374


In [39]:
X[0]

array([0.51948052, 0.09225936, 0.05307035, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       1.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 1.        ,
       0.        , 0.        , 0.        , 1.        , 0.        ,
       1.        , 0.        , 0.        , 1.        , 1.        ,
       0.        , 0.        , 0.        , 1.        ])

In [40]:
test_user = np.array([0.51948052, 0.09225936, 0.05307035, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       1.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 1.        ,
       0.        , 0.        , 0.        , 1.        , 0.        ,
       1.        , 0.        , 0.        , 1.        , 1.        ,
       0.        , 0.        , 0.        , 1.        ]).reshape(1,-1)

logreg.predict(test_user) #the user WILL NOT BUY (1)

array([0])

In [41]:
print (classification_report (y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.99      0.94     29966
           1       0.61      0.17      0.26      3943

    accuracy                           0.89     33909
   macro avg       0.76      0.58      0.60     33909
weighted avg       0.87      0.89      0.86     33909



In [ ]:
#Precision: Model's revelancy (87%) - of all the offers made, 87% users liked them.
#Recall: Model's completeness (89%) - of all the products users liked, 89% of those products were offered to them.

# Decision Trees

In [44]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
# from sklearn.tree import export_graphviz
# from sklearn.externals.six import StringIO 
# from IPython.display import Image 
# from pydot import graph_from_dot_data

# dot_data = StringIO()
# export_graphviz(dt, out_file=dot_data)
# (graph, ) = graph_from_dot_data(dot_data.getvalue())
# Image(graph.create_png())


In [45]:
y_pred = dt.predict(X_test)